In [54]:
import pandas as pd
from os.path import join
import numpy as np

In [55]:
dirname = "../data"
annot_file = join(dirname, "annotations.csv")
text_file = join(dirname, "data.csv")

df_txt = pd.read_csv(text_file)
df = pd.read_csv(annot_file)
# df = pd.merge(df_txt, df, on="text_id")
df.head()
df[df["text_id"] == 0]

,text_id,user_id,Pozytywne,Negatywne,Radość,Zachwyt,Inspiruje,Spokój,Zaskoczenie,Współczucie,...,Interesujący,Zrozumiały,Zgadzam się z tekstem,Wierzę w tę informację,"Potrzebuję więcej informacji, aby ocenić ten tekst",Czuję sympatię do autora,Obraża mnie,Może kogoś atakować / obrażać / lekceważyć,Mnie bawi/śmieszy?,Może kogoś bawić?
0,0,25,0,2,0,0,0,0,0,2,...,0,10,1,3,0,0,0,8,-1,2
1,0,4,8,3,0,6,8,3,2,0,...,0,4,7,3,7,0,0,8,0,0
2,0,24,6,8,3,2,0,0,4,8,...,0,7,3,3,1,2,6,7,0,0
3,0,33,2,8,2,2,2,2,6,7,...,0,2,0,0,7,0,8,8,0,0
4,0,6,5,5,6,3,3,3,0,0,...,2,10,7,9,0,3,0,3,0,0
5,0,3,0,8,0,0,0,0,10,0,...,0,10,0,0,0,0,0,10,0,4
6,0,27,0,9,0,0,0,0,4,10,...,0,0,-1,0,-1,0,5,9,0,0
7,0,13,8,0,8,4,2,4,1,1,...,3,9,7,7,7,7,-1,9,9,8
8,0,38,2,1,1,0,0,1,2,4,...,0,4,1,0,0,0,10,10,5,9
9,0,22,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9,0,3


In [56]:
label_columns = list(set(df.columns) - {"text_id", "user_id"})
df.shape

(31338, 28)

In [57]:
def major_vote(df, threshold=1):
    means = df[label_columns + ["text_id"]].groupby(["text_id"]).mean().reset_index()
    condition = means[label_columns] >= threshold
    means[label_columns] = np.where(condition, 1, 0)
    return means

def binarization(df):
    df[label_columns] = np.where(df[label_columns] > 2, 1, 0)
    return df

In [58]:
df_bin = binarization(df.copy())
df_bin.shape


(31338, 28)

In [59]:
all_annot = ((df[label_columns] == -1).sum(axis=1) > 0).sum()/df.shape[0]
all_annot

0.4337545471950986

In [66]:
dim_annot = (df[label_columns] <= 0).sum(axis=0)/df.shape[0]
dim_annot

Smutek                                                0.641362
Mnie bawi/śmieszy?                                    0.908418
Złość                                                 0.588168
Współczucie                                           0.687217
Negatywne                                             0.420161
Może kogoś bawić?                                     0.850405
Spokój                                                0.711788
Zachwyt                                               0.825803
Ironiczny                                             0.669634
Pozytywne                                             0.582903
Wulgarny                                              0.860106
Żenujący                                              0.775608
Interesujący                                          0.445721
Radość                                                0.762620
Inspiruje                                             0.752856
Obraża mnie                                           0

In [61]:
drop_columns = ["Czuję sympatię do autora", "Wierzę w tę informację", "Zgadzam się z tekstem"]
print(df_bin.columns)
df_bin.drop(columns=drop_columns, inplace=True)
label_columns = list(set(label_columns) - set(drop_columns))

Index(['text_id', 'user_id', 'Pozytywne', 'Negatywne', 'Radość', 'Zachwyt',
       'Inspiruje', 'Spokój', 'Zaskoczenie', 'Współczucie', 'Strach', 'Smutek',
       'Wstręt', 'Złość', 'Ironiczny', 'Żenujący', 'Wulgarny', 'Polityczny',
       'Interesujący', 'Zrozumiały', 'Zgadzam się z tekstem',
       'Wierzę w tę informację',
       'Potrzebuję więcej informacji, aby ocenić ten tekst',
       'Czuję sympatię do autora', 'Obraża mnie',
       'Może kogoś atakować / obrażać / lekceważyć', 'Mnie bawi/śmieszy?',
       'Może kogoś bawić?'],
      dtype='object')


In [62]:
df_bin_m = df_bin.merge(df_txt, on="text_id")
df_bin_m.drop(columns=["user_id"], inplace=True)
df_bin_m.head(2)

,text_id,Pozytywne,Negatywne,Radość,Zachwyt,Inspiruje,Spokój,Zaskoczenie,Współczucie,Strach,...,Wulgarny,Polityczny,Interesujący,Zrozumiały,"Potrzebuję więcej informacji, aby ocenić ten tekst",Obraża mnie,Może kogoś atakować / obrażać / lekceważyć,Mnie bawi/śmieszy?,Może kogoś bawić?,text
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,"Bardzo ładnie i praktycznie, każdy góral kocha..."
1,0,1,1,0,1,1,1,0,0,0,...,0,0,0,1,1,0,1,0,0,"Bardzo ładnie i praktycznie, każdy góral kocha..."


In [63]:
avg = {}
print(df_bin.columns)
df_copy = df_bin.copy()
avg = {"threshold": [], "rounded": [], "average": [], "lost_dimensions": []}
for th in range(0,101, 5):
    df_curr = major_vote(df_copy.copy(), threshold=th/100)
    avg_ = df_curr[label_columns].sum(axis=1).mean()
    lost = df_curr[label_columns].sum(axis=0).value_counts()
    lost_value = lost[0] if 0 in lost.index else 0
    avg["threshold"].append(th)
    avg["rounded"].append(round(avg_))
    avg["average"].append(avg_)
    avg["lost_dimensions"].append(lost_value)
df_avg = pd.DataFrame(data=avg)
df_avg.to_csv("averages_ones.csv", index=False)
df_avg

Index(['text_id', 'user_id', 'Pozytywne', 'Negatywne', 'Radość', 'Zachwyt',
       'Inspiruje', 'Spokój', 'Zaskoczenie', 'Współczucie', 'Strach', 'Smutek',
       'Wstręt', 'Złość', 'Ironiczny', 'Żenujący', 'Wulgarny', 'Polityczny',
       'Interesujący', 'Zrozumiały',
       'Potrzebuję więcej informacji, aby ocenić ten tekst', 'Obraża mnie',
       'Może kogoś atakować / obrażać / lekceważyć', 'Mnie bawi/śmieszy?',
       'Może kogoś bawić?'],
      dtype='object')


,threshold,rounded,average,lost_dimensions
0,0,23,23.000,0
1,5,15,15.369,0
2,10,13,12.818,0
3,15,11,10.704,0
4,20,9,9.216,0
5,25,8,7.821,0
6,30,7,6.580,0
7,35,5,5.457,0
8,40,5,4.592,0
9,45,4,3.849,0


In [64]:
# df_mv_m.to_csv("..\data\data_mv.csv", index=False, header=True)
# df_bin_m.to_csv("..\data\data_bin.csv", index=False, header=True)
